## Each student has personal set of questions

Google sheet with personal questions: https://docs.google.com/spreadsheets/d/16NfDeMJGCGsrsHYFV5qWC3iexrSdc93c-NHQvKka5K8/edit?usp=sharing

Every column corresponds to a single question, every row to a single student.

For example, Maksim Komiakov need to report questions 1.1, 1.2: 2.2, 2.3; 3.4, 3.1 etc.

## Submiting results

Google form to submit your answers: https://forms.gle/auZXVxfr4sk3cCmk8

Google form has fields for all questions, but you only need to answer **your** questions (from google sheet above).

Use your **skoltech email**. Fill your first and last names with **exactly same spelling** as in canvas system.

---

Every question has an information about the type of the answer, e.g.

> Observe top 10 observations (int)

here your answer must be a single **integer** number.

---

If your answer is a ``float number``, then it must be provided with **3 decimals after the floating point**, e.g. 1.234

---

If your answer is a ``list of float or integer numbers or str``, then they should be reported in descending (alphabetical) order, without spacing, divided by a comma, e.g.:

10.453,9.112,5.001,5.000 - **Right**

10.453, 9.112, 5.001, 5.000 - **Wrong**

---

Part of the tasks, e.g. Q19.1-5, Q20.1-5, do not have corresponding fields in the google form. They are **not optional** and they will be graded manually from your .ipynb file.

---

If you have any questions regarding this Home Assignment, ask them via telegram chat, topic 'HW1'.

# Assignment 1. House Pricing.
by Anvar Kurmukov

---

By the end of this task you will be able to manipulate huge tabular data:
1. Compute different column's statistics (min, max, mean, quantiles etc.);
2. Select observations/features by condition/index;
3. Create new non-linear combinations of the columns (feature engineering);
4. Perform automated data cleaning;

and more.

---

For those who are not familiar with `pandas` we recommend these (alternative) tutorials:

1. Single notebook, covers basic pandas functionality (starting with renaming columns ending with using map, apply etc) ~ 30 short examples with links on videos https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb . Highly recommended for everyone. (about 1-3 hours to go through)

2. https://github.com/guipsamora/pandas_exercises/ 11 topics covering all essential functionality with excersises (with solutions).

This task will be an easy ride after these tutorials.

---

We are using house sale price data from King County, Wahington, USA. This dataset is in public domain and can be obtained from Kaggle: https://www.kaggle.com/harlfoxem/housesalesprediction

You need to place `kc_house_data.csv` file in the same directory as this notebook.

In [1]:
import pandas as pd
import time
import numpy as np
import random
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split

random.seed(42)
np.random.seed(42)

In [2]:
class DataScience:


    def missing_zero_values_table(self, df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
            columns={0: 'Zero Values', 1: 'Missing Values', 2: '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:, 1] != 0].sort_values(
            '% of Total Values', ascending=False).round(1)
        print("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"
                                                                                                       "There are " + str(
            mz_table.shape[0]) +
              " columns that have missing values.")

        return mz_table

    
    def sns_draw(self, data, parametr, bins=30):
        fig = plt.figure(figsize=(15, 6), dpi= 80)
        grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.2)
        ax_main = fig.add_subplot(grid[:-1, :-1])
        ax_right = fig.add_subplot(grid[:-1, -1])  
        sns.distplot(data[parametr], ax=ax_main)
        sns.boxplot(data[parametr], ax=ax_right)
        ax_right.grid(axis='y')
        ax_main.set(title='График '+parametr)
        ax_main.axvline(np.mean(data[parametr]),color='g', linestyle='--', label='Среднее')
        ax_main.axvline(np.median(data[parametr]),color='g', linestyle='-', label='Медиана')
        ax_main.legend()
        fig.show()
    
    
    def plot_feature_importance(self,importance,names,model_type):

        feature_importance = np.array(importance)
        feature_names = np.array(names)

        #Create a DataFrame using a Dictionary
        data={'feature_names':feature_names,'feature_importance':feature_importance}
        fi_df = pd.DataFrame(data)

        #Sort the DataFrame in order decreasing feature importance
        fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

        #Define size of bar plot
        plt.figure(figsize=(10,8))
        #Plot Searborn bar chart
        sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
        #Add chart labels
        plt.title(model_type + 'FEATURE IMPORTANCE')
        plt.xlabel('FEATURE IMPORTANCE')
        plt.ylabel('FEATURE NAMES')
        
    def fit(self,train_features, train_actuals):
        for name in models.keys():
            est = models[name]
            est_params = params[name]
            gscv = GridSearchCV(estimator=est, param_grid=est_params, cv=5)
            gscv.fit(train_features, train_actuals)
            train_predictions = gscv.predict(features_train)
            predictions = gscv.predict(features_test)
            result_list.append({
            'model': name,
            'train_mae_score': mean_absolute_error(target_train, train_predictions),
            'test_mae_score': mean_absolute_error(target_test, predictions),
            'best_params': gscv.best_params_
            })
            
    
    def dataset_info(self,data):
        data.info()
        display(data.head(10))
        display(data.describe())
        print('Количество дубликатов:', data.duplicated().sum())
    

            
ds = DataScience()

In [3]:
def checktype(obj):
        return bool(obj) and all(isinstance(elem, str) for elem in obj)

In [4]:
def list_answer(answers):
    '''
    answers -- list with any format
    
    print answers in HW1 format
    '''
    if checktype(answers):
        answers.sort()
    else:
        answers.sort(reverse=True)
    for i in answers:
        if i != answers[-1]:
            print(i, end=',')
        else:
            print(i)

# 1. Loading data

As always in Data Science you are starting with making nice cup of tea (or coffee). Your next move is to load the data:

- Start with loading `house_data.csv` file using `pd.read_csv()` function.
- You may also want to increase maximal displayed pandas columns: set `pd.options.display.max_columns` to 30
- Print top 10 observations in the table. `.head()`
- Print last 10 observations in the table. `.tail()`
- Print all the data columns names using method `.columns`
- Print data size (number of rows and columns). This is the `.shape` of the data.

*Almost* every python has a `head` and a `tail` just as DataFrames do.

## Load the data

In [5]:
df = pd.read_csv('kc_house_data.csv', encoding='latin')

Lets see our data info.

In [6]:
ds.dataset_info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


Количество дубликатов: 0


We see, that bedrooms has int type, and bathrooms - float, but it isn't mistake - there houses with 2.5 bathrooms. Also there is houses with 3.5 floor.

no duplicates

## Observe top 10 observations (int)

In [7]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Q1.1 What is the price of a house with `id` == 7237550310?

In [8]:
int(df['price'].loc[df['id'] == 7237550310])



1225000

## Q1.5 What is the yr_renovated of a house with id == 6414100192?

In [9]:

int(df['yr_renovated'].loc[df['id'] == 6414100192])

1991

## Observe last 10 observations (int)

In [10]:

df.tail(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
21603,7852140040,20140825T000000,507250.0,3,2.50,2270,5536,2.0,0,0,...,8,2270,0,2003,0,98065,47.5389,-121.881,2270,5731
21604,9834201367,20150126T000000,429000.0,3,2.00,1490,1126,3.0,0,0,...,8,1490,0,2014,0,98144,47.5699,-122.288,1400,1230
21605,3448900210,20141014T000000,610685.0,4,2.50,2520,6023,2.0,0,0,...,9,2520,0,2014,0,98056,47.5137,-122.167,2520,6023
21606,7936000429,20150326T000000,1007500.0,4,3.50,3510,7200,2.0,0,0,...,9,2600,910,2009,0,98136,47.5537,-122.398,2050,6200
21607,2997800021,20150219T000000,475000.0,3,2.50,1310,1294,2.0,0,0,...,8,1180,130,2008,0,98116,47.5773,-122.409,1330,1265
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287
21612,1523300157,20141015T000000,325000.0,2,0.75,1020,1076,2.0,0,0,...,7,1020,0,2008,0,98144,47.5941,-122.299,1020,1357


## Q2.2 How many bedrooms has a house with `id` == 291310100?

In [11]:
int(df['bedrooms'].loc[df['id'] == 291310100])

3

## Q2.1 What is the price of a house with `id` == 263000018?

In [12]:
int(df['price'].loc[df['id'] == 263000018])

360000

## Increase maximal displayed columns

In [13]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Observe top 10 observations again
Is there any new columns displayed?

In [14]:
df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


We didn't see columns between 'view' and 'grade'. Now we can see column 'condition'.

## Print all the columns/features names (int)

In [15]:
print(df.keys())

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'], dtype='object')


## Q3.4 How many columns have `rooms` in their names?

In [16]:
k = 0
for i in df.keys():
    if 'rooms' in i:
        k+=1
print(k)

2


## Q3.3 How many columns assosiated with house location (except `zipcode`) are in the data?
lat - Lattitude  
long - Longitude  

I dont know how to calculate this in python, so i will calculate it manually.

In [17]:
print(2)

2


## Print data size (int)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

## Q4.1 How many observations are in the data?

In [19]:
df['id'].count()

21613

## Q4.2 How many features are in the data?

In [20]:
len(df.keys())

21

# 2. Basic data exploration

Lets do some basics:
- `.count()` number of not NaN's in every column.
- Is there any missing values in the data?
- Count number of unique values in every column `.nunique()`. 
- What does this tells you about the features, which are most likely categorical and which are most likely numerical?
- Use pandas `.describe()` to display basic statistic about the data.
- Use pandas `.value_counts()` to count number of unique values in a specific column.
- Use pandas `.min()`, `.max()`, `.mean()`, `.std()` to display specific statistics about the data.
- Use pandas `.dtypes` field to display data types in columns.

**Hint**
You could use `.sort_index()` or `.sort_values()` to sort the result of `.value_counts()`

In [21]:
df.nunique()

id               21436
date               372
price             4028
bedrooms            13
bathrooms           30
sqft_living       1038
sqft_lot          9782
floors               6
waterfront           2
view                 5
condition            5
grade               12
sqft_above         946
sqft_basement      306
yr_built           116
yr_renovated        70
zipcode             70
lat               5034
long               752
sqft_living15      777
sqft_lot15        8689
dtype: int64

## Display number of not NaN's in every column (int)

There is no NaN's.

In [22]:
for i in df.keys():
    print('Unique values in '+str(i)+' : ',  df[i].isnull().sum())

Unique values in id :  0
Unique values in date :  0
Unique values in price :  0
Unique values in bedrooms :  0
Unique values in bathrooms :  0
Unique values in sqft_living :  0
Unique values in sqft_lot :  0
Unique values in floors :  0
Unique values in waterfront :  0
Unique values in view :  0
Unique values in condition :  0
Unique values in grade :  0
Unique values in sqft_above :  0
Unique values in sqft_basement :  0
Unique values in yr_built :  0
Unique values in yr_renovated :  0
Unique values in zipcode :  0
Unique values in lat :  0
Unique values in long :  0
Unique values in sqft_living15 :  0
Unique values in sqft_lot15 :  0


## Q5.1 How many NA values are in the `floors` column?

In [23]:
print(df['floors'].isnull().sum())

0


## Q5.2 How many NA values are in the `grade` column?

In [24]:
print(df['grade'].isnull().sum())

0


## Count number of unique values in every column (int)

In [25]:
df.nunique()

# Q6.1 How many unique values are in the `bedrooms` column?
# Q6.2 How many unique values are in the `grade` column?
# Q6.3 How many unique values are in the `yr_renovated` column?
# Q6.4 How many unique values are in the `bathrooms` column?
# Q6.5 How many unique values are in the `long` column?


id               21436
date               372
price             4028
bedrooms            13
bathrooms           30
sqft_living       1038
sqft_lot          9782
floors               6
waterfront           2
view                 5
condition            5
grade               12
sqft_above         946
sqft_basement      306
yr_built           116
yr_renovated        70
zipcode             70
lat               5034
long               752
sqft_living15      777
sqft_lot15        8689
dtype: int64

## Q6.1 How many unique values are in the `bedrooms` column?

In [26]:
df['bedrooms'].nunique()

13

## Q6.3 How many unique values are in the `yr_renovated` column?

In [27]:
df['yr_renovated'].nunique()

70

In [28]:
# Count frequency of the values in different columns (list of ints in descending order)
# You could select a column using same syntax as for selecting a key from a dictionary: `data[colname]`
# Q7.1 For every unique `floors` value give its number of occurences.
# Q7.2 For every unique `condition` value give its number of occurences.
# Q7.3 For every unique `bedrooms` value give its number of occurences.
# Q7.4 For every unique `grade` value give its number of occurences.
# Q7.5 For every unique `view` value give its number of occurences.


## Q7.1 For every unique `floors` value give its number of occurences.

There should be more elegant way to show answer in needed format(maybe by split() and sep()), but I wasn't able to find it :(
Please - show it, if you can.

First I will show 2d array with number of floors and number of flats

In [29]:
df['floors'].value_counts()

1.0    10680
2.0     8241
1.5     1910
3.0      613
2.5      161
3.5        8
Name: floors, dtype: int64

This is answer in needed format.

In [30]:
uniq_floors = list((df['floors'].value_counts()))
list_answer(uniq_floors)

10680,8241,1910,613,161,8


## Q7.4 For every unique `grade` value give its number of occurences.

Again for Q7.4 

In [31]:
df['grade'].value_counts()

7     8981
8     6068
9     2615
6     2038
10    1134
11     399
5      242
12      90
4       29
13      13
3        3
1        1
Name: grade, dtype: int64

In [32]:
uniq_grades = list((df['grade'].value_counts()))
list_answer(uniq_grades)

8981,6068,2615,2038,1134,399,242,90,29,13,3,1


## Display basic data statistics using .describe()

In [33]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


## Display some column statistics (list of floats, rounded up to 3 digits, e.g. 1.234)

In [34]:
# Display some column statistics (list of floats, rounded up to 3 digits, e.g. 1.234)

# Q8.1 What are the max, min, mean and the std of the `floors` column?
# Q8.2 What are the max, min, mean and the std of the `bedrooms` column?
# Q8.3 What are the max, min, mean and the std of the `sqft_living` column?
# Q8.4 What are the max, min, mean and the std of the `pice` column?
# Q8.5 What are the max, min, mean and the std of the `long` column?


## Q8.3 What are the max, min, mean and the std of the `sqft_living` column?

In [35]:
print(round(df['sqft_living'].max(), 3),round(df['sqft_living'].min(), 3), 
      round(df['sqft_living'].mean(), 3),round(df['sqft_living'].std(), 3), sep = ',')

13540,290,2079.9,918.441


Format of answer should be in descending order?

In [36]:
i = 'sqft_living'
qua_e_t = [round(df[i].max(), 3),round(df[i].min(), 3), 
      round(df[i].mean(), 3),round(df[i].std(), 3)]
list_answer(qua_e_t)

13540,2079.9,918.441,290


## Q8.4 What are the max, min, mean and the std of the `price` column?

In [37]:
i = 'price'
qua_e_f = [round(df[i].max(), 3),round(df[i].min(), 3), 
      round(df[i].mean(), 3),round(df[i].std(), 3)]
print(round(df[i].max(), 3),round(df[i].min(), 3), 
      round(df[i].mean(), 3),round(df[i].std(), 3))


7700000.0 75000.0 540088.142 367127.196


Sorted one

In [38]:
list_answer(qua_e_f)

7700000.0,540088.142,367127.196,75000.0


## Display data types of all columns (int)

First of all, lets count all datatypes

In [39]:
types = df.dtypes.value_counts()
types
# Q9.1 How many columns have `object` data type?
# Q9.2 How many columns have `int64` data type?
# Q9.3 How many columns have `float64` data type?

# Display data types of all columns (list of str)
# Q9.4 What are the columns with dtype == `float64`?
# Q9.5 What are the columns with dtype == `int64`?


int64      15
float64     5
object      1
dtype: int64

In [40]:
list_answer(list(types))

15,5,1


Lets see, what idex we have here.

In [41]:
types.index

Index([int64, float64, object], dtype='object')

## Q9.2 How many columns have `int64` data type?

In [42]:
types[0]

15

## Q9.5 What are the columns with dtype == `int64`?

In [43]:
col_int = df.select_dtypes(include=['int64']).keys()
col_int

Index(['id', 'bedrooms', 'sqft_living', 'sqft_lot', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'sqft_living15', 'sqft_lot15'], dtype='object')

sorted one:

In [44]:
#list_answer(col_int)
col_int = list(col_int)
list_answer(col_int)
#k = col_int.sorted()

bedrooms,condition,grade,id,sqft_above,sqft_basement,sqft_living,sqft_living15,sqft_lot,sqft_lot15,view,waterfront,yr_built,yr_renovated,zipcode


# 3. Data selection

In pandas.DataFrame you could select

1. Row/s by position (integer number \[0 .. number of rows - 1\]) `.iloc` or by DataFrame.index `.loc`:
```
data.loc[0]
data.loc[5:10]
data.iloc[0]
data.iloc[5:10]
```
*Though, this is probably the worst way to manipulate rows.*

2. Columns by name
```
data[columname]
```
3. Row/s and columns
```
data.loc[10, columname]
data.iloc[10, columname]
```
4. Using boolean mask
```
mask = data[columname] > value
data[mask]
```
You could combine multiple conditions using `&` or `|` (and, or)

```
cond1 = data[columname1] > value1
cond2 = data[columname2] > value2
data[cond1 & cond2]
```
5. Using queries `.query()`:
```
value = 5
data.query("columname > value")
```
You could combine multiple conditions using `and`, `or`

```
data.query("(columname1 > value1) and (columname2 > value2)")
```
and others. See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html for more examples.

*Remember to use different quotation marks " or ' for columnname inside a query.*


In [45]:
data = df

In [46]:
# setting DataFrame index to be an `id` column, now .loc and .iloc will have different behavior
data.index = data.id

# dropping `id` column, since now it is an index
data.drop('id', axis=1, inplace=True)

# sort data by index for clarity
data.sort_index(inplace=True)

let's see, what we have:

In [47]:
data.head(5)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
1000102,20150422T000000,300000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,98002,47.3262,-122.214,2060,7316
1000102,20140916T000000,280000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,98002,47.3262,-122.214,2060,7316
1200019,20140508T000000,647500.0,4,1.75,2060,26036,1.0,0,0,4,8,1160,900,1947,0,98166,47.4444,-122.351,2590,21891
1200021,20140811T000000,400000.0,3,1.00,1460,43000,1.0,0,0,3,7,1460,0,1952,0,98166,47.4434,-122.347,2250,20023
2800031,20150401T000000,235000.0,3,1.00,1430,7599,1.5,0,0,4,6,1010,420,1930,0,98168,47.4783,-122.265,1290,10320


## Select rows by position (int) 

In [48]:
# Select rows by position (int) 

# Q10.1 How many bedrooms have a house on row 777?
# Q10.2 When was built a house on row 9999?
# Q10.3 How many floors have a house on row 1337?
# Q10.4 How many bathrooms have a house on row 314?
# Q10.5 What is the grade of a house on row 2718?


## Q10.1 How many bedrooms have a house on row 777?

In [49]:
data['bedrooms'].iloc[777]

3

## Q10.3 How many floors have a house on row 1337?

In [50]:
int(data['floors'].iloc[1337])

1

## Select rows by index (int)

In [51]:
# Select rows by index (int)

# Q11.1 How many times the house with index 1000102 were sold?
# Q11.2 What is the price of the house with index 9842300095?
# Q11.3 When was built the house with index 104510440?
# Q11.4 What is the condition of a house with index 252000300?
# Q11.5 What is the living area (in square feets) of the house with index 1225069038?


## Q11.1 How many times the house with index 1000102 were sold?

In [52]:
data['date'].loc[1000102].count()

2

## Q11.4 What is the condition of a house with index 252000300?

In [53]:
data['condition'].loc[252000300]

3

## Using mask or .query syntax select rows/columns (int)

In [54]:
# Using mask or .query syntax select rows/columns (int)

# Q12.1 How many houses were built during American Great Depression (1929–1939)? Including both start and end year.
# Q12.2 When was built the only house with basement area = 1024 sqft?
# Q12.3 How many houses are with the highest possible grade?
# Q12.4 When was built a house with maximal number of bedrooms?
# Q12.5 How many houses were sold for 256000 dollars?


## Q12.1 How many houses were built during American Great Depression (1929–1939)? Including both start and end year.

In [55]:
mask1 = data['yr_built'] > 1928
mask2 = data['yr_built'] < 1940


In [56]:
data['date'][mask1 & mask2].count()

644

## Q12.4 When was built a house with maximal number of bedrooms?

In [57]:
mask = data['bedrooms'] == data['bedrooms'].max()
int(data['yr_built'][mask])

1947

In [58]:
# Using mask or .query syntax select rows/columns (int)

# Q13.1 How many houses with the waterfront (=1) were built duroing Nixon's presidency (1969—1974)? Including both start and end year.
# Q13.2 How many houses, built before first human in space (<1961), have high condition (=5)?
# Q13.3 How many houses are with 6 bedrooms and less than 2000 sqft living area?
# Q13.4 What was the price of a house with 5 bathrooms, built in 1998 and graded with 10 score?
# Q13.5 How many floors has a house built in 1999 with 5 bedrooms and 3400 sqft living area?


## Using mask or .query syntax select rows/columns (int)

Date column is an object, but should be datetime, lets fix it.  
Also, we need only year, lets make new column.

In [59]:
data['date_y'] = data['date'].apply(lambda x: pd.to_datetime(x).strftime('%Y'))

In [60]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_y
id,,,,,,,,,,,,,,,,,,,,,
1000102,20150422T000000,300000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,98002,47.3262,-122.214,2060,7316,2015
1000102,20140916T000000,280000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,98002,47.3262,-122.214,2060,7316,2014
1200019,20140508T000000,647500.0,4,1.75,2060,26036,1.0,0,0,4,8,1160,900,1947,0,98166,47.4444,-122.351,2590,21891,2014
1200021,20140811T000000,400000.0,3,1.00,1460,43000,1.0,0,0,3,7,1460,0,1952,0,98166,47.4434,-122.347,2250,20023,2014
2800031,20150401T000000,235000.0,3,1.00,1430,7599,1.5,0,0,4,6,1010,420,1930,0,98168,47.4783,-122.265,1290,10320,2015


## Q13.2 How many houses, built before first human in space (<1961), have high condition (=5)?

In [61]:
mask1 = data['condition'] == 5
mask2 = data['yr_built'] < 1961


In [62]:
data['date'][mask1 & mask2].count()

1153

## Q13.1 How many houses with the waterfront (=1) were built duroing Nixon's presidency (1969—1974)? Including both start and end year.

In [63]:
mask1 = data['waterfront'] == 1
mask2 = data['yr_built'] < 1975
mask3 = data['yr_built'] > 1968

In [64]:
data['date'][mask1 & mask2 & mask3].count()

6

In [65]:
# Using mask or .query syntax select rows/columns and compute simple statistics (float)

# Q14.1 What was the average (sold) price of a houses built in the year of Cuban Missile Crisis (1962)?
# Q14.2 What was the most expensive house sold in the last decade of 20th centuary?
# Q14.3 What was the least expensive house sold in the last decade of 20th centuary?
# Q14.4 What is the median number of bathrooms in houses with grade above 9 (10 and more)?
# Q14.5 What is the median grade of houses with most popular zipcode value?


## Using mask or .query syntax select rows/columns and compute simple statistics (float)

## Q14.2 What was the most expensive house sold in the last decade of 20th centuary?

In [66]:
print(data['yr_built'].max(), data['yr_built'].min())

2015 1900


No information for last decade of 20th centuary...

In [67]:
mask1 = data['yr_built'] < 2000
mask2 = data['yr_built'] > 1989
int(list(data[mask1 & mask2].sort_values(by=['price'], ascending = False).head(1).index)[0])

7558700030

## Q14.1 What was the average (sold) price of a houses built in the year of Cuban Missile Crisis (1962)?

In [68]:
mask1 = data['yr_built'] == 1962
round(data['price'][mask1].mean(), 3)

436715.314

# 4. Creating new columns


Creating new column of pandas.DataFrame is as easy as:
```
data['new_awesome_column'] = [] 
```
that's it. But such a column is relatively useless. 
Typically, you would compute something new based on existing data and save it in a new column. 
For example one might want to compute total area of the house as a sum of all `sqft_` columns, or
create a boolean column of whether the house has `grade` > 2 or anything else:

```
data['total_area'] = data[col1] + data[col2] + ...
data['high_value'] = data[col] > 5
```

Pandas also provides another powerfull tool: `.apply`, `.map()`, `.applymap()` methods (they are kinda the same, but not quite). https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas .
They allow you to *apply* some function to every value in the column/s (row-wise) or row (column-wise) or cell (element-wise). For example, same computations of `total_area` and `high_value` using `.apply()`:
```
data['total_area'] = data[[col1, col2, col3]].apply(sum, axis=1)

```
you are not restricted to existent functions, `.apply()` accepts any function (including lambda functions):

```
data['total_area'] = data[[col1, col2, col3]].apply(lambda x: x[0]+x[1]+x[2], axis=1)
```
or ordinary python function (if this it should have complex behaviour):
```
def _sum(x):
    total = 0
    for elem in x:
        total += elem
    return total
    
data['total_area'] = data[[col1, col2, col3]].apply(_sum, axis=1) 
```
Many pandas methods has `axis` parameter `axis=0` refers to rows, `axis=1` refers to columns.

*Warning. You should never use `for` loops to sum numerical elements from the container.*

---


## Create `was_renovated` column. Bool column (0, 1) indicating whether the house was renovated.

Let's use astype to make 0 and 1

In [69]:
data['was_renovated'] = (data['yr_renovated']  != 0).astype(int)

Lets's check!

In [70]:
data[['was_renovated', 'yr_renovated']].head(20)

,was_renovated,yr_renovated
id,,
1000102,0,0
1000102,0,0
1200019,0,0
1200021,0,0
2800031,0,0
3600057,1,2013
3600072,0,0
3800008,0,0
5200087,0,0


In [71]:
# Create new columns using the old ones (new column in your DataFrame)

# Q15.1 Create a `sqft_tot_area` column (sum of all columns with `sqft_` prefix) using any method above
# Q15.2 Create a new column `sqm_tot_area` using `sqft_tot_area` and the fact that 1 foot = 0.3048 meters
# Q15.3 Create a new column `sqm_aver_floor_area` by dividing total area (in meters) by number of floors
# Q15.4 Create a new column `price_cat` by splitting a `price` into 5 ([1..5]) distinct intervals: 0 < x <=20%,
# 20% < x <= 40%, ... 80% < x <= 100% percentiles. You could use `.quantile()` to compute percentiles.
# Q15.5 Create a new bool column `high_class` it is True if the house has grade >= 9 and condition >= 4




## Create new columns using the old ones (new column in your DataFrame)

## Q15.1 Create a `sqft_tot_area` column (sum of all columns with `sqft_` prefix) using any method above

In [72]:
data['sqft_tot_area']=np.zeros(len(data))
k = []
for i in df.keys():
    if ('sqft_' in i) and(i != 'sqft_tot_area'):
        data['sqft_tot_area']+=data[i]
        k.append(i)

Here I check, that all needed keys were found:

In [73]:
k

['sqft_living',
 'sqft_lot',
 'sqft_above',
 'sqft_basement',
 'sqft_living15',
 'sqft_lot15']

Lets check new column:

In [74]:
data[['sqft_tot_area', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15',
      'sqft_lot15']].sample(5)

,sqft_tot_area,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15
id,,,,,,,
321059091,81786.0,1560,31299,1560,0,2460,44907
1250200605,11260.0,1190,3600,1190,0,1680,3600
1705400550,13130.0,1700,4165,1700,0,1400,4165
7312400075,13070.0,910,4800,910,0,1450,5000
8964800445,36635.0,3110,14872,3110,0,3110,12433


## Q15.2 Create a new column `sqm_tot_area` using `sqft_tot_area` and the fact that 1 foot = 0.3048 meters

In [75]:
data['sqm_tot_area'] = round(data['sqft_tot_area']*0.3048, 3)

Lets check manually

In [76]:
data[['sqm_tot_area', 'sqft_tot_area']].sample(5)

,sqm_tot_area,sqft_tot_area
id,,
7626200235,4489.704,14730.0
7195800009,8622.182,28288.0
8833510190,8168.030,26798.0
3832711040,6760.464,22180.0
1952000150,8738.921,28671.0


And calculate average

In [77]:
(data['sqm_tot_area']/data['sqft_tot_area']).mean()

0.30479999997897567

## Q15.3 Create a new column `sqm_aver_floor_area` by dividing total area (in meters) by number of floors

In [78]:
data['sqm_aver_floor_area'] = round(data['sqm_tot_area']/data['floors'], 3)

In [79]:
data[['sqm_aver_floor_area', 'sqm_tot_area', 'floors']]

,sqm_aver_floor_area,sqm_tot_area,floors
id,,,
1000102,3588.868,7177.735,2.0
1000102,3588.868,7177.735,2.0
1200019,16653.358,16653.358,1.0
1200021,20785.226,20785.226,1.0
2800031,4484.421,6726.631,1.5
...,...,...,...
9842300095,2585.923,3878.885,1.5
9842300485,5038.954,5038.954,1.0
9842300540,3772.205,3772.205,1.0


## Q15.4 Create a new column `price_cat` by splitting a `price` into 5 ([1..5]) distinct intervals: 0 < x <=20%, 20% < x <= 40%, ... 80% < x <= 100% percentiles. You could use `.quantile()` to compute percentiles.

To check, I will make column with percent rank

In [80]:
data['q'] = data['price'].rank(pct='true')

Here new column:

In [81]:
data['price_cat'] = df['price'].rank(pct='true').apply(lambda x: int(x*100//20)+1)

Lets check:

In [82]:
data[['price_cat', 'q']]

,price_cat,q
id,,
1000102,2,0.208393
1000102,1,0.169088
1200019,4,0.751955
1200021,3,0.405635
2800031,1,0.082959
...,...,...
9842300095,2,0.339865
9842300485,2,0.368251
9842300540,2,0.285546


## Q15.5 Create a new bool column `high_class` it is True if the house has grade >= 9 and condition >= 4

In [83]:
data['high_class'] = ((data['grade']  >= 9) & (data['condition']  >= 4))

In [84]:
data[['high_class', 'grade', 'condition']].sample(10)

,high_class,grade,condition
id,,,
3271800910,True,10,5
7501000080,False,10,3
5490210670,False,7,4
1823059223,False,7,3
7000100635,False,6,3
1685800100,False,10,3
9211010440,False,8,3
3323069084,False,8,3
623049232,False,5,3


In [85]:
# Using mask or .query syntax select rows/columns (float)

# Q16.1 What is the average price of the house of the high_class(=True)?
# Q16.2 What is the average total_area (in meters) of the house from highest price category?
# Q16.3 What is the maximal number of floors amongst houses with the lowest price category?
# Q16.4 What is the most frequent zipcode amongst houses with the lowest price category?
# Q16.5 What is the minimal number of bathrooms in houses with high_class=True?


## Using mask or .query syntax select rows/columns (float)

## Q16.3 What is the maximal number of floors amongst houses with the lowest price category?

In [86]:
mask1 = data['price_cat'] == 1
int(round(data['floors'][mask1].max(), 3))

3

## Q16.1 What is the average price of the house of the high_class(=True)?

In [87]:
mask1 = data['high_class'] == True
round(data['price'][mask1].mean(), 3)

1146802.328

# 5. Basic date processing

You figure out that column `date` is to harsh for you, so you decided to convert it to a more plausible format:

- Use pandas method `to_datetime()` to convert the date to a good format.
- Exctract `year`, `month`, `day` and `weekday` from your new date column. Save them to separete columns.
- How many columns has your data now?
- Drop column `date`, remember to set `inplace` parameter to True.


**Hint** for datetime formatted date you could extract the `year` as follow:
```
data.date.dt.year
```

Very often date could be a ridiculously rich feature, sometimes it is holidays that matters, sometimes weekends, sometimes some special days like **black friday**. 

Learn how to work with date in Python!

In [88]:
# Create new columns based on `date` column

# Q17.1 Convert date to datetime format
# Q17.2 Extract and store `year`
# Q17.3 Extract and store `month`
# Q17.4 Extract and store `day`
# Q17.5 Extract and store `weekday`
# Q17.6 Create a new column `house_age_10` - the age of the house in full decades (e.g. 9 year old house - 0, 21 year old house - 2),
# using `yr_built` and 'year' columns


## Q17.1 Convert date to datetime format

In [89]:
data['date'] = pd.to_datetime(data['date'])

In [90]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_y,was_renovated,sqft_tot_area,sqm_tot_area,sqm_aver_floor_area,q,price_cat,high_class
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000102,2015-04-22,300000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,98002,47.3262,-122.214,2060,7316,2015,0,23549.0,7177.735,3588.868,0.208393,2,False
1000102,2014-09-16,280000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,98002,47.3262,-122.214,2060,7316,2014,0,23549.0,7177.735,3588.868,0.169088,1,False
1200019,2014-05-08,647500.0,4,1.75,2060,26036,1.0,0,0,4,8,1160,900,1947,0,98166,47.4444,-122.351,2590,21891,2014,0,54637.0,16653.358,16653.358,0.751955,4,False
1200021,2014-08-11,400000.0,3,1.00,1460,43000,1.0,0,0,3,7,1460,0,1952,0,98166,47.4434,-122.347,2250,20023,2014,0,68193.0,20785.226,20785.226,0.405635,3,False
2800031,2015-04-01,235000.0,3,1.00,1430,7599,1.5,0,0,4,6,1010,420,1930,0,98168,47.4783,-122.265,1290,10320,2015,0,22069.0,6726.631,4484.421,0.082959,1,False


## Q17.2 Extract and store `year`

In [91]:
data['year'] = data.date.dt.year

In [92]:
data[['year', 'date']].head(5)

,year,date
id,,
1000102,2015,2015-04-22
1000102,2014,2014-09-16
1200019,2014,2014-05-08
1200021,2014,2014-08-11
2800031,2015,2015-04-01


## Q17.3 Extract and store `month`

In [93]:
data['month'] = data.date.dt.month

In [94]:
data[['month', 'date']].head(5)

,month,date
id,,
1000102,4,2015-04-22
1000102,9,2014-09-16
1200019,5,2014-05-08
1200021,8,2014-08-11
2800031,4,2015-04-01


## Q17.4 Extract and store `day`

In [95]:
data['day'] = data.date.dt.day

In [96]:
data[['day', 'date']].head(5)

,day,date
id,,
1000102,22,2015-04-22
1000102,16,2014-09-16
1200019,8,2014-05-08
1200021,11,2014-08-11
2800031,1,2015-04-01


## Q17.5 Extract and store `weekday`

In [97]:
data['weekday'] = data.date.dt.weekday
data[['weekday', 'date']].head(5)

,weekday,date
id,,
1000102,2,2015-04-22
1000102,1,2014-09-16
1200019,3,2014-05-08
1200021,0,2014-08-11
2800031,2,2015-04-01


## Q17.6 Create a new column `house_age_10` - the age of the house in full decades (e.g. 9 year old house - 0, 21 year old house - 2),

In [98]:
current_year = datetime.datetime.now().year
data['house_age_10'] = data['yr_built'].apply(lambda x: int((current_year-x)//10))
data[['house_age_10', 'yr_built']].sample(5)

,house_age_10,yr_built
id,,
2788400090,6,1960
4112100165,2,2001
3586500700,6,1957
1062100100,5,1965
8732131200,4,1978


In [99]:
data['house_age_10'] = (data['year'] - data['yr_built'])//10
data[['house_age_10', 'yr_built', 'year']].sample(5)

,house_age_10,yr_built,year
id,,,
3735901080,6,1946,2015
9164100105,10,1909,2015
123039336,7,1939,2014
9126100813,0,2014,2014
2254100090,0,2012,2015


## Drop column `date`

In [100]:
# Drop column `date`
data = data.drop(columns=['date'])

In [101]:
# Find some date related information from the data (int)

# Q18.1 What is the most popular selling weekday?
# Q18.2 What is the most popular selling month?
# Q18.3 What is the least popular selling weekday?
# Q18.4 What is the median age of the house (on a first available sold date)? (float)
# Q18.5 How many houses were sold on America's Independence Day (July, 4)?


## Q18.5 How many houses were sold on America's Independence Day (July, 4)?

In [102]:
mask1 = data['day'] == 4
mask2 = data['month'] == 7
data['zipcode'][mask1 & mask2].count()

2

## Q18.1 What is the most popular selling weekday?

In [103]:
data.weekday.value_counts().index[0]

1

# 6. Groupby
From the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

By “group by” we are referring to a process involving one or more of the following steps:

    - Splitting the data into groups based on some criteria.
    - Applying a function to each group independently.
    - Combining the results into a data structure.
    
---
`.groupby()` is one of the most powerfull tool for feature engineering. Very often it is used to group object with the same categorical characteristics and compute some statistics (e.g. mean, max, etc.) of a their numerical characteric. 

Instead of computing average area of houses with high grade you could compute average areas of the
houses for every grade in a single command:

```
data.groupby('grade')['sqm_tot_area'].mean()
```

You could also make multi-column groups:

```
data.groupby(['weekday','grade'])['price'].min()
```
next, you could compute multiple aggregation functions:
```
data.groupby(['weekday','grade'])['price'].agg([min, max])
```

instead of using built-in functions you could compute custom functions using apply:
```
import numpy as np
data.groupby(['condition','grade'])['bathrooms'].apply(lambda x: np.quantile(x, .5))
```

and the coolest thing now is that you can map the results of groupby back on your DataFrame!
```
gp = data.groupby(['condition'])['bathrooms'].median()
data['gp_feature'] = data['condition'].map(gp)
```
Now, if some house has `condition == 2`, its `gp_feature` will be equal to the median number of 
bathrooms amongst all houses with `condition == 2`.

Read more examples in the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [104]:
# Create some groupby features

# Q19.1 `price_by_class` groupby `high_class` and compute median `price`.
# Q19.2 `price_by_year` groupby `year` and compute median price.
# Q19.3 `price_by_weekday` groupby `weekday` and compute median price.
# Q19.4 `area_by_price` groupby `price_cat` and compute average `sqft_living`.
# Q19.5 `floors_by_age` groupby `floors` and compute average age of a house.

## Q19.2 `price_by_year` groupby `year` and compute median price.

In [105]:
gp = round(data.groupby('year')['price'].mean(), 3)
data['price_by_year'] = data['year'].map(gp)

In [106]:
gp

year
2014    539181.428
2015    541988.992
Name: price, dtype: float64

In [107]:
data.sample(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_y,was_renovated,sqft_tot_area,sqm_tot_area,sqm_aver_floor_area,q,price_cat,high_class,year,month,day,weekday,house_age_10,price_by_year
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1994200375,601150.0,2,2.00,1660,5200,1.0,0,0,5,7,1120,540,1906,0,98103,47.6871,-122.334,1260,5160,2014,0,14940.0,4553.712,4553.712,0.710961,4,False,2014,12,3,2,10,539181.428
7214400005,663500.0,2,1.00,1310,5200,1.0,0,0,3,7,910,400,1946,0,98115,47.6784,-122.304,1320,4794,2015,0,13934.0,4247.083,4247.083,0.768473,4,False,2015,3,13,4,6,541988.992
2344300220,1100000.0,4,3.50,2210,7597,1.0,0,0,4,9,1550,660,1977,2006,98004,47.5816,-122.197,2370,8811,2014,1,23198.0,7070.750,7070.750,0.944293,5,True,2014,7,14,0,3,539181.428
3601200017,175000.0,4,2.50,1780,6000,2.0,0,0,3,7,1780,0,1991,0,98198,47.3828,-122.302,1630,6000,2014,0,17190.0,5239.512,2619.756,0.019456,1,False,2014,7,2,2,2,539181.428
1424059130,247500.0,3,0.75,1300,72309,1.0,0,0,3,6,680,620,1950,1987,98006,47.5670,-122.124,3080,8395,2015,1,86384.0,26329.843,26329.843,0.102554,1,False,2015,3,18,2,6,541988.992


## Q19.1 `price_by_class` groupby `high_class` and compute median `price`.

In [108]:
gp = round(data.groupby('high_class')['price'].mean(), 3)
data['price_by_class'] = data['high_class'].map(gp)

In [109]:
data[['high_class', 'price_by_class']].sample(10)

,high_class,price_by_class
id,,
3902100150,False,514702.366
7202260040,False,514702.366
3629990180,False,514702.366
4139440100,False,514702.366
1424069110,False,514702.366
7221400285,False,514702.366
2787250190,False,514702.366
7212680860,False,514702.366
9407001320,False,514702.366


In [110]:
# Create some other groupby features
# for this task check out this answer:
# https://stackoverflow.com/questions/47913343/how-to-groupby-and-map-by-two-columns-pandas-dataframe

# Q20.1 `n_houses_zipcode` groupby `zipcode` and count number of occurences of every unique zipcode
# Q20.2 `n_houses_yr_built` groupby `yr_built` and count number of houses built in each year
# Q20.3 `price_by_yr_month_`(median, std) groupby `year`, `month` and compute median and std `price`.
# Q20.4 `price_by_grade_age_`(median, std) groupby `grade`, `house_age` and compute median and std `price`.
# Q20.5 `living_by_cond_`(median, std) groupby `waterfront`, `view`, `condition` and compute median and std `sqft_living`.


## Q20.5 `living_by_cond_`(median, std) groupby `waterfront`, `view`, `condition` and compute median and std `sqft_living`.

In [111]:
gp = data.groupby(['waterfront', 'view', 'condition'])['sqft_living'].median()
gp = gp.reset_index(level=['waterfront', 'view', 'condition']).rename(columns = {'sqft_living':'living_by_cond_median'})
data = pd.merge(data, gp, on =['waterfront', 'view', 'condition'], how = 'left')
gp = data.groupby(['waterfront', 'view', 'condition'])['sqft_living'].std()
gp = gp.reset_index(level=['waterfront', 'view', 'condition']).rename(columns = {'sqft_living':'living_by_cond_std'})
data = pd.merge(data, gp, on =['waterfront', 'view', 'condition'], how = 'left')
data.sample(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_y,was_renovated,sqft_tot_area,sqm_tot_area,sqm_aver_floor_area,q,price_cat,high_class,year,month,day,weekday,house_age_10,price_by_year,price_by_class,living_by_cond_median,living_by_cond_std
3942,1700000.0,4,3.5,4070,336283,2.0,0,0,3,11,4070,0,2006,0,98027,47.4780,-122.038,3020,44613,2015,0,392056.0,119498.669,59749.334,0.983806,5,False,2015,3,12,3,0,541988.992,514702.366,1910.0,875.297176
15128,372500.0,3,1.5,1180,12324,1.0,0,0,3,7,800,380,1981,0,98074,47.6337,-122.032,1280,11371,2014,0,27335.0,8331.708,8331.708,0.352589,2,False,2014,6,5,3,3,539181.428,514702.366,1910.0,875.297176
20376,611000.0,3,2.5,2134,1984,2.5,0,0,3,8,2134,0,2008,0,98027,47.5402,-122.027,2170,1984,2014,0,10406.0,3171.749,1268.700,0.719706,4,False,2014,5,16,4,0,539181.428,514702.366,1910.0,875.297176
2898,237200.0,3,1.5,1220,9000,1.0,0,0,4,7,1220,0,1968,0,98042,47.3642,-122.109,1220,9472,2015,0,22132.0,6745.834,6745.834,0.086152,1,False,2015,4,21,1,4,541988.992,514702.366,1760.0,730.099838
13126,355000.0,4,3.0,2590,7213,2.0,0,0,3,8,2590,0,2001,0,98001,47.2609,-122.289,2550,6800,2014,0,21743.0,6627.266,3313.633,0.321612,2,False,2014,12,8,0,1,539181.428,514702.366,1910.0,875.297176


## Q20.3 `price_by_yr_month_`(median, std) groupby `year`, `month` and compute median and std `price`.

In [112]:
gp = data.groupby(['year', 'month'])['price'].median()
gp = gp.reset_index(level=['year', 'month']).rename(columns = {'price':'price_by_yr_month_median'})
data = pd.merge(data, gp, on =['year', 'month'], how = 'left')
gp = data.groupby(['year', 'month'])['price'].std()
gp = gp.reset_index(level=['year', 'month']).rename(columns = {'price':'price_by_yr_month_std'})
data = pd.merge(data, gp, on =['year', 'month'], how = 'left')
data.sample(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_y,was_renovated,sqft_tot_area,sqm_tot_area,sqm_aver_floor_area,q,price_cat,high_class,year,month,day,weekday,house_age_10,price_by_year,price_by_class,living_by_cond_median,living_by_cond_std,price_by_yr_month_median,price_by_yr_month_std
4047,230000.0,3,1.75,1720,9125,1.0,0,0,4,7,1140,580,1981,0,98092,47.3173,-122.181,1120,7506,2015,0,21191.0,6459.017,6459.017,0.075695,1,False,2015,3,5,3,3,541988.992,514702.366,1760.0,730.099838,450000.0,355051.196554
18230,475000.0,4,3.50,3660,14401,2.0,0,0,3,10,2660,1000,1994,0,98023,47.3076,-122.396,2780,10653,2014,0,35154.0,10714.939,5357.470,0.538912,3,False,2014,10,10,4,2,539181.428,514702.366,1910.0,875.297176,446900.0,399243.064472
3877,360000.0,3,1.75,1810,7200,1.0,0,0,5,7,1030,780,1959,0,98146,47.4993,-122.364,1950,8384,2015,0,21154.0,6447.739,6447.739,0.332138,2,False,2015,5,11,0,5,541988.992,514702.366,1800.0,755.383338,455000.0,414821.964244
2588,415000.0,4,3.50,3040,7125,2.0,0,1,3,8,2240,800,2002,0,98178,47.4920,-122.225,2220,7800,2014,0,23225.0,7078.980,3539.490,0.430713,3,False,2014,9,25,3,1,539181.428,514702.366,2540.0,1116.481347,450000.0,372531.666484
25,837700.0,5,2.75,3010,12611,2.0,0,0,3,10,3010,0,1994,0,98052,47.6960,-122.102,2890,9456,2014,0,30977.0,9441.790,4720.895,0.880165,5,False,2014,6,17,1,2,539181.428,514702.366,1910.0,875.297176,465000.0,388260.134140


# 7. Building a regression model

> You do not need to normalize data for tree models; for linear/knn models this step is essential.

> Remember, that not all of the features in the table are numeric, some of them might be viewed as categorical.

> You may create or drop **any** features you want, except for the features which use `price` (e.g. average price of a house with 5 bedrooms).

## Q21 Drop all generated features which used price column, e.g. price_by_year, price_cat.

In [113]:
data = data.drop(columns=['price_by_year'])
data = data.drop(columns=['price_cat'])
data = data.drop(columns=['price_by_class'])
data = data.drop(columns=['price_by_yr_month_median'])
data = data.drop(columns=['price_by_yr_month_std'])

In [114]:
data['high_class'] = data['high_class'].astype(int)

In [115]:
data.sample(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_y,was_renovated,sqft_tot_area,sqm_tot_area,sqm_aver_floor_area,q,high_class,year,month,day,weekday,house_age_10,living_by_cond_median,living_by_cond_std
4842,362300.0,3,2.50,2430,15264,2.0,0,0,3,8,2430,0,1997,0,98030,47.3805,-122.208,2260,10416,2014,0,32800.0,9997.440,4998.720,0.335770,0,2014,6,23,0,1,1910.0,875.297176
9473,827000.0,4,2.50,3230,12100,1.0,0,0,3,9,1870,1360,1977,0,98006,47.5620,-122.146,2670,10200,2014,0,31430.0,9579.864,9579.864,0.875792,0,2014,6,26,3,3,1910.0,875.297176
11434,525000.0,3,1.00,1550,6840,1.5,0,0,3,7,1550,0,1918,0,98103,47.6572,-122.335,2370,4560,2014,0,16870.0,5141.976,3427.984,0.609032,0,2014,10,24,4,9,1910.0,875.297176
3748,263000.0,3,1.75,1410,8100,2.0,0,0,3,7,1410,0,1985,0,98038,47.3617,-122.046,1560,8100,2014,0,20580.0,6272.784,3136.392,0.133438,0,2014,6,30,0,2,1910.0,875.297176
7881,573500.0,3,1.00,2200,21450,1.0,0,0,4,9,1600,600,1952,0,98166,47.4527,-122.372,1880,11250,2015,0,38980.0,11881.104,11881.104,0.676121,1,2015,4,2,3,6,1760.0,730.099838


## Q22 Split your data into train and test parts.
# How many records (rows) do you have in train and test tables? (list of int)?
# Use sklearn.model_selection.train_test_split with test_size=0.33 and random_state=42

In [116]:
data_l = data
data_l = data_l.drop(columns=['zipcode', 'date_y', 'was_renovated', 'sqft_tot_area', 'sqm_aver_floor_area', 'q', 'high_class', 
                             'living_by_cond_median', 'living_by_cond_std'])

In [117]:
data_encoded = pd.get_dummies(data_l)
X = data_encoded.drop(columns=['price'])  # Features
y = data_encoded['price']  # Target variable
X.fillna(X.mean(), inplace=True)

In [118]:
y.mean()

540088.1417665294

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [120]:
an = [len(X_train), len(X_test)]
list_answer(an)

14480,7133


## Q23.1 Use linear regression with l2 regularization (Ridge regression)

In [121]:
min_max_scaler = MinMaxScaler().fit(X_test)
data_o_n = min_max_scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [122]:
y_test.mean()

539146.7163886162

In [123]:
len(X_train), len(y_train)

(14480, 14480)

In [124]:
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,sqm_tot_area,year,month,day,weekday,house_age_10
16197,3,1.00,1560,7552,1.0,0,0,4,6,910,650,1948,0,47.7552,-122.327,1200,8152,6103.315,2014,7,7,0,6
16993,2,1.00,900,8000,1.0,0,0,4,6,900,0,1954,0,47.4217,-122.317,1240,8000,5803.392,2014,8,12,1,6
17538,5,2.75,3110,6621,2.0,0,0,3,9,3110,0,2006,0,47.5430,-121.888,3550,7953,7420.051,2015,4,20,0,0
5761,3,1.00,1180,8474,1.5,0,0,3,7,1180,0,1956,0,47.7416,-122.327,1180,7200,5856.427,2014,10,22,2,5
18921,3,2.50,2320,57063,1.0,0,0,4,9,1790,530,1979,0,47.6163,-122.056,3290,7314,22039.174,2014,10,29,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,4,3.00,1960,5160,1.0,0,0,3,7,1170,790,2001,0,47.5313,-122.360,1960,5160,4937.760,2015,2,10,1,1
21575,3,1.00,1790,3876,1.5,0,0,5,7,1090,700,1904,0,47.5750,-122.288,1360,4080,3930.701,2014,7,16,2,11
5390,3,1.00,1030,60720,1.5,0,0,3,5,1030,0,1912,0,47.3750,-122.166,1330,10342,22692.970,2014,9,16,1,10
860,2,1.00,830,6017,1.0,0,0,4,6,830,0,1954,0,47.7466,-122.334,1340,6040,4589.374,2014,5,16,4,6


In [125]:
# Create linear regression object
regr = Ridge(alpha=1.0)

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [126]:
y_pred.mean()

541551.3575083136

In [127]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse).round(3)
print('mse:', mse)
print('rmse:', rmse)
print('Variance score: %.2f' % r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

mse: 35911308284.76182
rmse: 189502.792
Variance score: 0.71
124010.88098456911


In [128]:
len(X_test), len(y_test), len(y_pred)

(7133, 7133, 7133)

## Q23.3 Use k nearest neighbours regression

In [129]:
best_model = KNeighborsRegressor(
   n_neighbors=10, 
   weights='distance',
   algorithm='auto',
   leaf_size=30,
   metric='euclidean',
   metric_params=None,
   n_jobs=4
)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse).round(3)
print('mse:', mse)
print('rmse:', rmse)
print('Variance score: %.2f' % r2_score(y_test, y_pred))

mse: 56640075100.86266
rmse: 237991.754
Variance score: 0.54


In [130]:
# Create a predictive regression model of a house price.

# Q23.1 Use linear regression with l2 regularization (Ridge regression)
# Q23.2 Use decision tree regression
# Q23.3 Use k nearest neighbours regression

In [131]:
# Use grid search to select optimal hyperparamters of your models. 

# Q24.1 Alpha for a ridge regression
# Q24.2 Depth for the tree
# Q24.3 Number of neighbours for the knn

## Q24.1 Alpha for a ridge regression

In [132]:
def model_eval(model, X_train, y_train, X_test, y_test):
    
    start = time.time()    
    model.fit(X_train, y_train)
    training_time = time.time() - start
    
    start = time.time()  
    y_pred = model.predict(X_test)
    predict_time = time.time() - start

    return round(training_time, 3), round(predict_time, 3), round(r2_score(y_test, y_pred), 3), round(mean_squared_error(y_test, y_pred)**0.5, 3), round(mean_squared_error(y_test, y_pred), 3)

In [133]:
parameters = {'alpha':np.linspace(1, 8, 20), }
scoring = {"MSE": 'neg_mean_squared_error', "R2": 'r2'}
# define the model/ estimator
model = Ridge()

# define the grid search
Ridge_reg= GridSearchCV(model, parameters, scoring='r2' ,cv=5, n_jobs = -1, verbose=False)

#fit the grid search
Ridge_reg.fit(X_train, y_train)


print(Ridge_reg.best_estimator_)

Ridge(alpha=4.684210526315789)


In [134]:
res_r = model_eval(Ridge_reg.best_estimator_, X_train, y_train, X_test, y_test)
y_pred = Ridge_reg.best_estimator_.predict(X_train)
mse_train = round(mean_squared_error(y_train, y_pred), 3)
r2_train = round(r2_score(y_train, y_pred), 3)
pd.DataFrame([res_r], columns=['training_time','predict_time','r2_score', 'rmse','mse'])

,training_time,predict_time,r2_score,rmse,mse
0,0.011,0.004,0.707,189542.035,3.592618e+10


## Q24.3 Number of neighbours for the knn

In [135]:
parameters = {'n_neighbors':np.arange(1, 15, 1)}
scoring = {"MSE": 'neg_mean_squared_error', "R2": 'r2'}
# define the model/ estimator
model = KNeighborsRegressor(weights='distance',
   algorithm='auto',
   leaf_size=30,
   metric='euclidean',
   metric_params=None,
   n_jobs=4)

# define the grid search
KNN_reg= GridSearchCV(model, parameters, scoring='neg_mean_squared_error' ,cv=5, n_jobs = -1, verbose=False)

#fit the grid search
KNN_reg.fit(X_train, y_train)


print(KNN_reg.best_estimator_)

KNeighborsRegressor(metric='euclidean', n_jobs=4, n_neighbors=12,
                    weights='distance')


In [136]:
res_n = model_eval(KNN_reg.best_estimator_, X_train, y_train, X_test, y_test)
y_pred = KNN_reg.best_estimator_.predict(X_train)
mse_train_k = round(mean_squared_error(y_train, y_pred), 3)
r2_train_k = round(r2_score(y_train, y_pred), 3)
print('R2:', res_n[2])

R2: 0.539


In [137]:
mse_train_k, r2_train_k

(0.227, 1.0)

In [138]:
pd.DataFrame([res_n], columns=['training_time','predict_time','r2_score', 'rmse','mse'])

,training_time,predict_time,r2_score,rmse,mse
0,0.004,0.133,0.539,237808.064,5.655268e+10


In [139]:
# Compute train and test mean squared error for your best models (list of float).

# Q25.1 Train, test MSE using linear regression with l2 regularization
# Q25.2 Train, test MSE using decision tree regression
# Q25.3 Train, test MSE using k nearest neighbours regression

## Q25.1 Train, test MSE using linear regression with l2 regularization

In [140]:
q25_1 = [mse_train, res_r[4]]
list_answer(q25_1)

43365400547.588,35926183048.241


## Q25.3 Train, test MSE using k nearest neighbours regression

In [141]:
res_n

(0.004, 0.133, 0.539, 237808.064, 56552675475.62)

In [142]:
q25_3 = [mse_train_k, res_n[4]]
list_answer(q25_3)

56552675475.62,0.227


In [143]:
# Compute train and test R^2 for your best models (list of float).

# Q26.1 Train, test R^2 using linear regression with l2 regularization
# Q26.2 Train, test R^2 using decision tree regression
# Q26.3 Train, test R^2 using k nearest neighbours regression

## Q26.1 Train, test R^2 using linear regression with l2 regularization 

In [144]:
q26_1 = [r2_train, res_r[2]]
list_answer(q26_1)

0.707,0.692


## Q26.3 Train, test R^2 using k nearest neighbours regression

In [145]:
q26_3 = [r2_train_k, res_n[2]]
list_answer(q26_3)

1.0,0.539


## Q27 Which features have largest (by absolute value) weight in your linear model (top 5 features)? (list of str).

In [150]:
# Q27 Which features have largest (by absolute value) weight in your linear model (top 5 features)? (list of str).
reg = Ridge_reg.best_estimator_
reg.fit(X_train, y_train)
d = {'weight': reg.coef_, 'name': list(X_train.keys())}
df = pd.DataFrame(data=d)
df['weight'] = abs(df['weight'])
q_27 = list(df.sort_values(by=['weight'], ascending = False).head(5)['name'])
list_answer(q_27)

grade,lat,long,view,waterfront


In [147]:
df

,weight,name
0,42871.996130,bedrooms
1,40587.639051,bathrooms
2,343.629454,sqft_living
3,167.239019,sqft_lot
4,3813.094777,floors
5,543902.994131,waterfront
6,53188.680913,view
7,34918.451769,condition
8,93809.314273,grade
9,189.453551,sqft_above


In [148]:
df.sort_values(by=['weight'], ascending = False)

,weight,name
13,559732.881292,lat
5,543902.994131,waterfront
14,112506.293754,long
8,93809.314273,grade
6,53188.680913,view
0,42871.996130,bedrooms
1,40587.639051,bathrooms
7,34918.451769,condition
18,33749.422614,year
22,20965.741751,house_age_10


# Make sure your .ipynb is linearly executable
# Kernel -> Restart & Run All -> No ERROR cells

In [149]:
# Q28 Save your .ipynb file: FirstName_SecondName_HA1.ipynb, you will be asked to upload it into the google form.